In [ ]:
!pip install -q -U google-generativeai

### Setup your API key

Before you can use the Gemini API, you must first obtain an API key. If you don't already have one, create a key with one click in Google AI Studio.

<a class="button button-primary" href="https://makersuite.google.com/app/apikey" target="_blank" rel="noopener noreferrer">Get an API key</a>

In Colab, add the key to the secrets manager under the "🔑" in the left panel. Give it the name `GOOGLE_API_KEY`.

Once you have the API key, pass it to the SDK. You can do this in two ways:

* Put the key in the `GOOGLE_API_KEY` environment variable (the SDK will automatically pick it up from there).
* Pass the key to `genai.configure(api_key=...)`

In [ ]:
# Used to securely store your API key
from google.colab import userdata

api_key = userdata.get('GOOGLE_API_KEY')

In [ ]:
import google.generativeai as genai
genai.configure(api_key=api_key)
model = genai.GenerativeModel('gemini-pro')

In [ ]:
def split_file(file_path):
    with open(file_path, 'r') as file:
        content = file.read()

    # Remove leading/trailing whitespace and split the content into sentences
    sentences = content.strip().split('.')

    # Remove empty sentences
    sentences = [sentence.strip() for sentence in sentences if sentence.strip()]

    # Calculate the number of sentences per chunk
    total_sentences = len(sentences)
    chunk_size = total_sentences // 10

    chunks = []
    start = 0
    for i in range(9):
        end = start + chunk_size
        chunk = sentences[start:end]
        chunks.append('. '.join(chunk) + '.')
        start = end

    # Add the remaining sentences to the last chunk
    last_chunk = sentences[start:]
    chunks.append('. '.join(last_chunk) + '.')

    return chunks

# Example usage
file_path = '/content/vinallama.txt'
chunks = split_file(file_path)

for i, chunk in enumerate(chunks, 1):
    print(f"Chunk {i}:")
    print(chunk)
    print()

Chunk 1:
In this technical report, we present VinaLLaMA, an open-source, state-of-the-art (SOTA) Large Language Model for the Vietnamese language, built upon LLaMA-2 with an additional 800 billion trained tokens. VinaLLaMA not only demonstrates fluency in Vietnamese but also exhibits a profound understanding of Vietnamese culture, making it a truly indigenous model. VinaLLaMA-7B-chat, trained on 1-million high quality synthetic samples, achieves SOTA results on key benchmarks, including VLSP, VMLU, and Vicuna Benchmark Vietnamese, marking a significant advancement in the Vietnamese AI landscape and offering a versatile resource for various applications. 1 Introduction 
The surge in Large Language Models (LLMs) such as ChatGPT and GPT-4 has significantly advanced the field of artificial intelligence (AI), particularly in language processing. In 2023, Vietnam’s AI sector witnessed a notable development with the introduction of several Vietnamese-centric LLMs, including BLOOMZ’s Vietcuna,

Summarization (Tóm Tắt) là một trong số những task khó nhất cho các LLM. Nó đỏi hỏi LLM có khả năng xử lí long context nhưng vẫn có khả năng làm retrieval trên text tốt. Vì vậy, đối vơi đại đa số các LLM, chúng ta sẽ làm prompt chaining để triển khai bài toán này

In [ ]:
question = """
Hãy tóm tắt văn bản: {context}.
Tôi chỉ cần nội dung tóm tắt, không cần thêm bất kì thứ gì khác.
"""

In [ ]:
def build_final_context(chunks):
    context = ""
    for index, chunk in enumerate(chunks):
        context +=  f"Context {index + 1}: " + chunk + "\n"
    return context

In [ ]:
context = build_final_context(chunks)
context

In [ ]:
%%time
response = model.generate_content(question.format(context=context))

CPU times: user 137 ms, sys: 15.6 ms, total: 153 ms
Wall time: 7.85 s


In [ ]:
response.text

'VinaLLaMA là một Mô hình Ngôn ngữ Lớn (LLM) tiên tiến, nguồn mở, dành riêng cho tiếng Việt, được xây dựng dựa trên LLaMA-2 với 800 tỷ token được đào tạo thêm. VinaLLaMA không chỉ thể hiện sự lưu loát tiếng Việt mà còn hiểu sâu sắc về văn hóa Việt Nam, biến nó trở thành một mô hình thực sự bản địa. VinaLLaMA-7B-chat, được đào tạo trên 1 triệu mẫu tổng hợp chất lượng cao, đạt kết quả SOTA trên các điểm chuẩn chính, bao gồm VLSP, VMLU và Vicuna Benchmark Vietnamese, đánh dấu một bước tiến đáng kể trong bối cảnh AI của tiếng Việt và cung cấp một nguồn đa năng cho nhiều ứng dụng khác nhau.'

In [ ]:
summarize_prompt = """
Sau đây là một nội dung được trích xuất của một văn bản, nhiệm vụ của bạn là hãy tóm tắt nó
---CONTEXT---
{context}
---END CONTEXT---
Hãy đưa ra tóm tắt của văn bản trên. Tôi chỉ cần phần tóm tắt, và không cần thêm bất kì thứ gì khác.
Tóm Tắt:
"""

final_ans_prompt = """
Dưới đây là các đoạn tóm tắt của từng đoạn nhỏ của một văn bản lớn.
---CONTEXT---
{context}
---END CONTEXT---
Dựa trên các đoạn trên, hãy đưa ra tóm tắt tổng của tất cả các đoạn trên. Tôi chỉ cần tóm tắt tổng, không cần thêm bất kì thứ gì khác.
Tóm tắt tổng:
"""

In [ ]:
summarize_chunks = []
for chunk in chunks:
  message = summarize_prompt.format(context=chunk)
  response = model.generate_content(message)
  summarize_chunks.append(response.text)
summarize_chunks

In [ ]:
context_new = build_final_context(summarize_chunks)
message = final_ans_prompt.format(context=context_new)
response = model.generate_content(message)
response.text


'VinaLLaMA-7B là một mô hình ngôn ngữ lớn tinh chỉnh hướng dẫn tiên tiến cho tiếng Việt, đạt hiệu suất vượt trội trong các điểm chuẩn về khả năng hiểu và tạo văn bản, nhờ dữ liệu tổng hợp được thiết kế cẩn thận. Các chiến lược tinh chỉnh được triển khai tốt với dữ liệu tổng hợp có tiềm năng nâng cao khả năng của các mô hình ngôn ngữ lớn, như được minh họa bởi sự cân bằng tối ưu giữa kích thước, tốc độ và hiệu suất của VinaLLaMA-7B-chat. Các bộ công cụ đánh giá như VMLU và Vicuna Benchmark Vietnamese cung cấp các cách đánh giá sáng tạo và toàn diện về hiệu suất của các mô hình này trên các nhiệm vụ đa dạng.'

In [ ]:
samples_retrieval = ['Hôm nay, giá Bitcoin đã đạt đến một đỉnh mới.',
           'Phở Việt Nam rất ngon, Hà Nội nổi tiếng với món Phở Bò.',
           'Phở là món ăn Việt Nam xuất hiện ở rất nhiều nơi trên thế giới.',
           'Một thị trường khó như Nhật cũng rất ấn tượng với món Phở Bò Việt Nam.']

context = build_final_context(samples_retrieval)
print(context)

Context 1: Hôm nay, giá Bitcoin đã đạt đến một đỉnh mới.
Context 2: Phở Việt Nam rất ngon, Hà Nội nổi tiếng với món Phở Bò.
Context 3: Phở là món ăn Việt Nam xuất hiện ở rất nhiều nơi trên thế giới.
Context 4: Một thị trường khó như Nhật cũng rất ấn tượng với món Phở Bò Việt Nam.



In [ ]:
sample_question = "Hãy giới thiệu về món Phở Bò Việt Nam"
refinement_prompt = """
Dưới đây là các nội dung được trích xuất từ Database:
---CONTEXT---
{context}
---END CONTEXT---
Nhiệm vụ của bạn là hãy viết lại những nội dung trên dựa trên câu hỏi dưới đây, hãy loại bỏ các nội dung không liên quan để tránh gây nhiễu loạn. Tôi chỉ cần phần viết lại của bạn, vui lòng không thêm bất kì thông tin nào khác.
Câu hỏi: {question}
"""

In [ ]:
sample_message = refinement_prompt.format(context=context, question=sample_question)
sample_response = model.generate_content(sample_message)
sample_response.text

'* Phở là món ăn Việt Nam xuất hiện ở rất nhiều nơi trên thế giới.\n* Thậm chí thị trường khó tính như Nhật Bản cũng rất thích món Phở Bò Việt Nam.'